In [ ]:
pip install --upgrade openai

In [37]:
import pandas as pd

# read xlsx file
df = pd.read_excel("/Users/johri/Projects/AIEarthHack/AI EarthHack Dataset.xlsx")
print(df.columns)
print("Before pre-processing: ", len(df))

# filter entries with less than 10 words in the solution column
df = df[df['solution'].str.split().str.len().gt(20)]

# filter entries with similar solution 
df = df.drop_duplicates(subset=['solution'])
print("After pre-processing: ", len(df))

Index(['id', 'problem', 'solution'], dtype='object')
Before pre-processing:  1300
After pre-processing:  1208


In [76]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="API_KEY",
)

def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        response_format={"type":"json_object"},
        messages=[
            {"role": "system", "content": "Provide output in valid JSON"},
            {"role": "user", "content": prompt}]
    )

    ret_json = response.choices[0].message.content.strip()
    
    return ret_json


## if using different version of openai
# import openai
# def chat_gpt(prompt):
#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": prompt}]
#     )
#     ret_json = json.dumps(response.choices[0].message.content.strip())
    
#     return ret_json


In [100]:
from ast import literal_eval

min_word_count = 10
min_bullet = 2
max_bullet = 5
key_features = "objective, relevance_to_circular_economy, feasibility, impact, technical_complexity, innovation, sustainability, market_potential, competitive_landscape, cost-effectiveness"
dict_keys = "'rating_objective', 'rating_relevance_to_circular_economy', 'rating_feasibility', 'rating_impact', 'rating_technical_complexity', 'rating_innovation', 'rating_sustainability', 'rating_market_potential', 'rating_competitive_landscape', 'rating_cost-effectiveness', 'explanation_objective', 'explanation_relevance_to_circular_economy', 'explanation_feasibility', 'explanation_impact', 'explanation_technical_complexity', 'explanation_innovation', 'explanation_sustainability', 'explanation_market_potential', 'explanation_competitive_landscape', 'explanation_cost-effectiveness'"
df_final = pd.DataFrame()
n = 10

for inx in range(0, 50, n):
    proposals = ""
    selected_df = df[inx:inx+n]
    for inx, row in selected_df.iterrows():
        problem_statement = row['problem'].strip()
        solution_statement = row['solution'].strip()
        proposals += f"id: {row['id']}\nproblem: {problem_statement}\nsolution: {solution_statement}\n"

    prompt = f"""Given are {n} proposals for a competition titled "Unlocking the Potential of Circular Economy".
                The solutions were sourced all over the world in various ways with circular economy applications to a wide array of industries, ranging from textiles to food waste management.
                Participants were asked about the problem their solution is meant to solve and describe the solution in their own words.
                These solutions must be presented before a human judge. To help the judge, extract the main idea from each proposal.
                Secondly, for each proposal provide comments and ratings using the following rubrics: {key_features}
                The comments for any proposal must not exceed {min_word_count} words. Limit to {min_bullet}-{max_bullet} bullet points. Rate each rubric from 0 to 100, with 0 being the worst and 100 being the best.
                Thirdly, generate a summary of the description of problem and solution for each propsal. Limit the summary to around 100 words.
                The output should be a single json as mentioned below: 
                {{
                    id1: {{
                        explanation_rubric1: ...,
                        rating_rubric1: ...,
                        explanation_rubric2: ...,
                        rating_rubric2: ...,
                        ...
                        summary: ...
                    }},
                    id2: {{
                        explanation_rubric1: ...,
                        rating_rubric1: ...,
                        explanation_rubric2: ...,
                        rating_rubric2: ...,
                        ...
                        summary: ... 
                        }},
                    ...
                }}

                Here, id1, id2, ... will be the respective proposal id.
                The rubric1, rubric2, ... will be: {dict_keys}.
                Ensure that the summary describes  the problem and solution clearly. 
                
                Use the following pairs of problem and solution enclosed in triple backticks:
                ```
                {proposals}
                ```
                """

    # print(prompt)
    response_json = chat_gpt(prompt)

    print(response_json)
    # json_object = literal_eval(json.loads(response_json))
    json_object = json.loads(response_json)

    json_df = pd.DataFrame(json_object).T
    df_final = pd.concat([df_final, json_df], ignore_index=True)

df_final

{
  "1": {
    "explanation_objective": "Mitigating construction waste through modular construction.",
    "rating_objective": 85,
    "explanation_relevance_to_circular_economy": "Modular construction embraces recycling and reuse, leading to circular economy practices.",
    "rating_relevance_to_circular_economy": 90,
    "explanation_feasibility": "Already proven feasibility in global markets.",
    "rating_feasibility": 80,
    "explanation_impact": "Decreases construction waste by 90% and time by 30-50%, optimizing environmental and financial efficiency.",
    "rating_impact": 95,
    "explanation_technical_complexity": "Low technical complexity.",
    "rating_technical_complexity": 70,
    "explanation_innovation": "Innovative approach towards circular economy in construction industry.",
    "rating_innovation": 85,
    "explanation_sustainability": "Reduces waste and conserves resources.",
    "rating_sustainability": 90,
    "explanation_market_potential": "Scalability and real-

,explanation_objective,rating_objective,explanation_relevance_to_circular_economy,rating_relevance_to_circular_economy,explanation_feasibility,rating_feasibility,explanation_impact,rating_impact,explanation_technical_complexity,rating_technical_complexity,...,rating_innovation,explanation_sustainability,rating_sustainability,explanation_market_potential,rating_market_potential,explanation_competitive_landscape,rating_competitive_landscape,explanation_cost-effectiveness,rating_cost-effectiveness,summary
0,Mitigating construction waste through modular ...,85,Modular construction embraces recycling and re...,90,Already proven feasibility in global markets.,80,Decreases construction waste by 90% and time b...,95,Low technical complexity.,70,...,85,Reduces waste and conserves resources.,90,Scalability and real-world application potenti...,75,Medium competitive landscape due to proven fea...,70,High cost-effectiveness due to substantial fin...,90,"Modular construction mitigates waste, decrease..."
1,Revolutionary approach for wind energy generat...,85,"Decreases reliance on fossil fuels, promoting ...",90,Feasibility indicated through under-utilized t...,80,"Creation of clean, quiet, and efficient energy...",95,Moderate technical complexity.,75,...,85,"Promotes sustainability through efficient, les...",90,Scalable concept without traditional windmills...,75,Medium competitive landscape due to innovative...,70,"High cost-effectiveness with efficient, less p...",85,"Revolutionary wind energy generation, clean, q..."
2,Reduction of carbon footprint and deforestatio...,80,"Reduces need for new book production, encourag...",85,Gamification and easy-to-use platform increase...,75,"Significantly reduces waste, deforestation, an...",90,Low technical complexity.,70,...,80,Contributes to sustainability by reducing book...,85,Scalable cultural shift towards resource value...,80,Medium competitive landscape due to gamificati...,75,"High cost-effectiveness for students, parents,...",85,"Book sharing program reduces waste, deforestat..."
3,Reduction of fashion industry pollution throug...,85,Transforms fashion industry from single-purcha...,90,"Dependent on location, culture, and income lev...",75,"Reduces production, transportation, and landfi...",95,Moderate technical complexity based on locatio...,70,...,85,"Encourages resource use efficiency, increases ...",90,Scalable model globally with potential culture...,80,Medium competitive landscape due to dependence...,75,High cost-effectiveness with continuous income...,90,"Garment rental service reduces waste, promotes..."
4,Promotion of modular electronic device model f...,85,Maximizes life and value of electronic compone...,90,Scalable concept for continuous technological ...,80,"Promotes resource use efficiency, cuts waste, ...",95,"Moderate technical complexity, scalable with t...",75,...,85,"Promotes resource use efficiency, value captur...",90,"Scalable model for global application, continu...",80,Medium competitive landscape due to scalable d...,75,High cost-effectiveness with sale of modules a...,85,Modular electronic device model maximizes valu...
5,Developing mobile app to digitize transactiona...,80,"Significantly reduces paper consumption, promo...",85,Feasible adoption due to trend of digital tech...,75,Improves environmental and financial efficienc...,90,Low technical complexity with existing infrast...,70,...,80,Contributes to environmental and financial ben...,85,"High scalability for businesses of all sizes, ...",80,Medium competitive landscape due to digital te...,75,High cost-effectiveness with substantial savin...,85,"Digitizing transactional evidence, reducing pa..."
6,Revolutionizing carsharing experience with Han...,85,"Reduces fuel consumption, and frustration, con...",90,Feasible implementation with potential for sea...,80,"Reduces extended driving distances, costs, and...",95,"Moderate technical complexity, feasible implem...",75,...,85,"Contributes to a greener future, re

In [119]:
df_final.fillna(0, inplace=True)
# df[['id']] = df[['id']].astype(int)
# result_df = pd.concat([df[['id']], df_final], axis=1) 
# result_df.to_csv("result.csv")


In [128]:
df_final['avg_rating'] = df_final[df_final.columns[df_final.columns.str.startswith('rating_')]].mean(axis=1)
df_final = df_final.sort_values(by=['avg_rating'], ascending=[False])

In [131]:
# Order of final sorted list of projects
df_final.to_csv("result.csv")
df_final

,explanation_objective,rating_objective,explanation_relevance_to_circular_economy,rating_relevance_to_circular_economy,explanation_feasibility,rating_feasibility,explanation_impact,rating_impact,explanation_technical_complexity,rating_technical_complexity,...,explanation_sustainability,rating_sustainability,explanation_market_potential,rating_market_potential,explanation_competitive_landscape,rating_competitive_landscape,explanation_cost-effectiveness,rating_cost-effectiveness,summary,avg_rating
37,Introducing a Smartphone Leasing and Hardware ...,92,"Promotes extended use, refurbishing, and recyc...",90,Feasible model with potential for long-term fi...,88,"Dramatically reduces e-waste, promotes sustain...",92,Challenges in technological refurbishing proce...,85,...,"Promotes sustainable technology consumption, e...",90,Potential for capturing a sustainable tech mar...,88,May face competition from traditional device o...,86,Cost-effective model with potential for long-t...,89,Introducing a Smartphone Leasing and Hardware ...,88.9
39,Implementing Eletronic Waste Return Dispensers...,90,"Promotes responsible e-waste disposal, resourc...",88,Feasible model with potential for high returns...,87,"Decreases hazardous e-waste, extends device li...",90,High initial investment for infrastructure and...,85,...,Promotes sustainable electronic waste manageme...,90,Potential for high revenue streams and seamles...,88,May face competition from traditional e-waste ...,86,Cost-effective model with potential for high r...,87,Implementing Eletronic Waste Return Dispensers...,88.0
14,Comprehensive circular economy model to tackle...,85,Encourages reduce-reuse-recycle paradigm and p...,90,Feasible with local SME collaborations and pre...,85,Scalable model with potential for global indus...,95,Moderate technical complexity with comprehensi...,80,...,Promotes sustainable fashion practices and loc...,95,Potential for network expansion and global ind...,85,Competitive advantage in sustainable fashion a...,85,Lucrative model with premium value added to Ge...,85,Comprehensive circular economy model to tackle...,87.5
34,Leveraging Chemical Leasing to promote efficie...,92,Promotes efficient and value-based chemical us...,88,Feasible implementation with potential scalabi...,85,"Reduces chemical waste, limits environmental i...",90,Can require adjustments in traditional volume-...,82,...,"Promotes sustainable chemical use, reduced was...",88,Potential for adoption across industries and a...,89,May face competition from traditional chemical...,86,Cost-effective model with potential for long-t...,87,Leveraging Chemical Leasing to promote efficie...,87.2
23,Addresses energy consumption and environmental...,90,Integrates renewable energy generation and ene...,95,Feasible through BIPV technology and intellige...,85,Significant potential to reduce energy depende...,95,"Complex considerations for building design, en...",80,...,Promotes renewable energy integration and ener...,90,Appeals to eco-conscious consumers and busines...,85,Competition from traditional building and ener...,80,Potential cost savings and environmental benef...,80,Addresses energy consumption and environmental...,87.0
31,Proposing an Electronics Leasing System to red...,92,Encouraging reuse and controlled recycling of ...,88,Feasible with potential financial benefits for...,85,Significantly reducing e-waste generation and ...,90,May face technical complexities in device refu...,80,...,Promotes sustainable technology use and resour...,86,Potential for a profitable model that aligns w...,88,May face competition from traditional device o...,84,Cost-effective model with financial benefits f...,87,Proposing an Electronics Leasing System to red...,86.5
30,Introducing recycled plastic construction mate...,95,Applying circular economy principles to reduce...,90,Feasible with potential for global scalability...,85,"Reducing landfill usage, pollution from plasti...",92,Can be technically challenging d